In [1]:
!pip install optuna catboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir('/content/drive/MyDrive/Euron')

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [7]:
#train.drop(columns=['ID', 'Book-ID', 'Year-Of-Publication'], inplace=True)
#test.drop(columns=['ID', 'Book-ID', 'Year-Of-Publication'], inplace=True)
# location 전처리
# Location 분리
train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')
train['city'] = train['Location'].apply(lambda x: x.split(',')[0].strip())
train['state'] = train['Location'].apply(lambda x: x.split(',')[1].strip())
train['country'] = train['Location'].apply(lambda x: x.split(',')[2].strip())

train.drop('Location', axis=1, inplace=True)
train = train.replace('na', np.nan)
train = train.replace('', np.nan)

# Location 최빈값으로 대체
mode_city = train['city'].mode()[0]
mode_state = train['state'].mode()[0]
mode_country = train['country'].mode()[0]

# 결측값 대체
train['city'].fillna(mode_city, inplace=True)
train['state'].fillna(mode_state, inplace=True)
train['country'].fillna(mode_country, inplace=True)

<ipython-input-7-4af6f511e24b>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')


In [8]:
## Age 이상치 처리
# 이상치: 5세 이하 90세 초과
train.loc[(train['Age'] <= 5)|(train['Age'] > 90), 'Age'] = np.nan
train['Age'].isnull().sum() # 5세 이하 또는 90세 초과인 데이터 4497개
# 평균값으로 대체
train['Age'].mean() # Age (이상치 제거 후) 평균: 36.541634752034845
train['Age'] = train['Age'].fillna(train['Age'].mean())

## Age 범주화
# Age_group: 이상치 처리 & 5세 단위로 범주화
train['Age_group'] = pd.cut(train['Age'], bins = list(range(5, 91, 5)))

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder() #labelencoder함수를 가져온다.
encoder.fit(train['Age_group']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder.transform(train['Age_group'])
train['Age_group'] = digit_label # encoding 잘됐나?
#train['Age_group']

In [9]:
# 소문자 통일, 특수문자 제거 (종류: 15505 -> 15021)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

# 소문자 통일, 특수문자 제거, 띄어쓰기 제거 (종류: 15505 -> 14840)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [10]:
train['Book-Author'] = train['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [11]:
# 출판년도 그룹화
# 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
# train['Year-Of-Publication'] = train['Year-Of-Publication'].fillna(-1)

labels = ['Unknown', '-1800', '1800-1850', '1850-1900', '1900-1910', '1910-1920', '1920-1930', '1930-1940', '1940-1950','1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
bins = [-1, 0, 1800, 1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
train['Pub_gb'] = pd.cut(train['Year-Of-Publication'], bins, labels = labels,include_lowest = True)
train = train.drop(columns =['Year-Of-Publication'])
#train['Year-Of-Publication'] = pd.cut(train['Year-Of-Publication'], bins=bins, labels=labels)


encoder2 = LabelEncoder() #labelencoder함수를 가져온다.
encoder2.fit(train['Pub_gb']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder2.transform(train['Pub_gb'])
train['Pub_gb'] = digit_label # encoding 잘됐나?
#train['Pub_gb']

In [13]:
# Title 데이터 정리
import re # 정규표현 모듈

# 정규표현식으로 대소문자 통일, 특수문자 제거
train['Book-Title'] = [re.sub(r'[^0-9a-zA-Z:,]',  ' ',str(i)) for i in train['Book-Title']]
train['Main_Title'] = [i.split('  ')[0] for i in train['Book-Title']]
train['Sub_Title'] = [''.join(i.split('  ')[1:]) for i in train['Book-Title']]
train['Sub_Title'] = np.where(train['Sub_Title'] == '', 'No_SUB', train['Sub_Title'])

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   871393 non-null  object 
 1   User-ID              871393 non-null  object 
 2   Book-ID              871393 non-null  object 
 3   Book-Rating          871393 non-null  int64  
 4   Age                  871393 non-null  float64
 5   Location             871393 non-null  object 
 6   Book-Title           871393 non-null  object 
 7   Book-Author          871393 non-null  object 
 8   Year-Of-Publication  871393 non-null  float64
 9   Publisher            871393 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 66.5+ MB


In [17]:
y = train['Book-Rating']

X = train.drop(['Book-Rating','Book-ID','ID','Age','Year-Of-Publication'], axis=1)

In [18]:
kf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

cat_feature = ['User-ID', 'Location', 'Book-Title', 'Book-Author', 'Publisher']

In [19]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

    params = {
        'objective' :  'RMSE',
        'n_estimators':trial.suggest_int("n_estimators", 30, 10000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'random_state' : 42,
        'task_type' : "GPU",
        'verbose' : False
    }


    model = CatBoostRegressor(
        **params,
        cat_features=['User-ID', 'Location', 'Book-Title', 'Book-Author', 'Publisher']
    )
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 500, verbose = 2000)

    pred = model.predict(x_test)

    mae = mean_squared_error(y_test, pred, squared=False)


    return mae

In [20]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=10)

[I 2024-02-06 10:57:14,035] A new study created in memory with name: no-name-c028a63e-844e-48f0-a3e0-0782ac539a19
<ipython-input-19-d522cf9583d8>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-d522cf9583d8>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-d522cf9583d8>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'random_strength': trial.suggest_uniform('random_strength',10,50),


0:	learn: 3.8023702	test: 3.7998162	best: 3.7998162 (0)	total: 91.1ms	remaining: 9m 56s
bestTest = 3.262208492
bestIteration = 1381
Shrink model to first 1382 iterations.


[I 2024-02-06 10:59:15,045] Trial 0 finished with value: 3.2622083177068038 and parameters: {'n_estimators': 6543, 'od_wait': 1713, 'learning_rate': 0.06785102146166193, 'reg_lambda': 23.595066552160397, 'random_strength': 12.712364961420901, 'depth': 7, 'min_data_in_leaf': 23}. Best is trial 0 with value: 3.2622083177068038.
<ipython-input-19-d522cf9583d8>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-d522cf9583d8>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-d522cf9583d8>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0

0:	learn: 3.6256075	test: 3.6101660	best: 3.6101660 (0)	total: 80.2ms	remaining: 3m 31s
bestTest = 3.268983439
bestIteration = 173
Shrink model to first 174 iterations.


[I 2024-02-06 11:00:06,735] Trial 1 finished with value: 3.268983505262289 and parameters: {'n_estimators': 2640, 'od_wait': 1562, 'learning_rate': 0.3507451072925427, 'reg_lambda': 13.410291337947946, 'random_strength': 41.415371621899226, 'depth': 8, 'min_data_in_leaf': 7}. Best is trial 0 with value: 3.2622083177068038.
<ipython-input-19-d522cf9583d8>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-d522cf9583d8>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-d522cf9583d8>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. T

...

[I 2024-02-06 11:06:01,628] Trial 7 finished with value: 3.270324057623459 and parameters: {'n_estimators': 973, 'od_wait': 703, 'learning_rate': 0.5013135523601162, 'reg_lambda': 38.31690819562641, 'random_strength': 19.638164121886486, 'depth': 7, 'min_data_in_leaf': 19}. Best is trial 0 with value: 3.2622083177068038.
<ipython-input-19-d522cf9583d8>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-d522cf9583d8>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-d522cf9583d8>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. Thi

0:	learn: 3.4322838	test: 3.3935649	best: 3.3935649 (0)	total: 240ms	remaining: 8m 58s
bestTest = 3.291451355
bestIteration = 17
Shrink model to first 18 iterations.


[I 2024-02-06 11:07:42,656] Trial 8 finished with value: 3.291451028703455 and parameters: {'n_estimators': 2245, 'od_wait': 2174, 'learning_rate': 0.960771444418275, 'reg_lambda': 74.07416057120774, 'random_strength': 42.47224356571064, 'depth': 13, 'min_data_in_leaf': 27}. Best is trial 0 with value: 3.2622083177068038.
<ipython-input-19-d522cf9583d8>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-d522cf9583d8>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-d522cf9583d8>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. Th

0:	learn: 3.7663482	test: 3.7619088	best: 3.7619088 (0)	total: 22.5ms	remaining: 2m 46s
2000:	learn: 3.3313058	test: 3.2722558	best: 3.2722549 (1999)	total: 42.9s	remaining: 1m 56s
4000:	learn: 3.3281887	test: 3.2711966	best: 3.2711876 (3854)	total: 1m 25s	remaining: 1m 13s
6000:	learn: 3.3266522	test: 3.2709174	best: 3.2708632 (5536)	total: 2m 5s	remaining: 29.6s
bestTest = 3.270863213
bestIteration = 5536
Shrink model to first 5537 iterations.


[I 2024-02-06 11:10:02,006] Trial 9 finished with value: 3.2708631277095908 and parameters: {'n_estimators': 7415, 'od_wait': 1031, 'learning_rate': 0.15117195019521404, 'reg_lambda': 55.08679562506948, 'random_strength': 29.289962858690846, 'depth': 2, 'min_data_in_leaf': 29}. Best is trial 0 with value: 3.2622083177068038.


In [21]:
params=study.best_params
params['objective'] = 'RMSE'
params['random_state'] = 42
params['verbose'] = False
params['task_type'] = "GPU"
params['cat_features'] = ['User-ID', 'Location', 'Book-Title', 'Book-Author', 'Publisher']

In [22]:
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

Best Score: 3.2622083177068038
Best trial: {'n_estimators': 6543, 'od_wait': 1713, 'learning_rate': 0.06785102146166193, 'reg_lambda': 23.595066552160397, 'random_strength': 12.712364961420901, 'depth': 7, 'min_data_in_leaf': 23}


In [23]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [24]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)